### retrieved latest version of WGS-pipeline (main branch)
*NOTE:* 
- the repo is big (about 266MB), I found it was due to this directory: ".git/objects/pack/" - need to check why it is included and if it is ok to remove it; 
- the file rmlst.json is in the repo and is of no use to the user - I would remove it

In [3]:
!git clone https://www.github.com/itaysol/WGS-pipeline.git

### for test run, created subset of test samples

In [9]:
#!pip install pandas
import pandas as pd
user_config = pd.read_csv("WGS-pipeline/user_config_file_subset.csv")
user_config

,sample,id,R1Fastq,R2Fastq,specie,comparisonGroup,username
0,sample1,SRR11901822,fastq/SRR11901822_1.fastq.gz,fastq/SRR11901822_2.fastq.gz,Metamycoplasma hominis,1,1
1,sample2,SRR11901823,fastq/SRR11901823_1.fastq.gz,fastq/SRR11901823_2.fastq.gz,Metamycoplasma hominis,1,1


### running snakemake using the subset of samples
*NOTE:* 
- before running, I needed to change the kraken db folder location twice in the Snakefile - lines 80 and 93
- this kraken db folder location should be defined either as a variable in the top of the Snakefile or as a variable in a config file

In [ ]:
!snakemake --use-conda --cores 2 --config file "user_config_file_subset.csv"

### first execution of snakemake failed after one sample was assembled
*NOTE:*
- not sure why snakemake run failed after sample2 completed?
- it may be that the cgMLST rule ran before the sample1 assembly ran?
- need to check that all assemblies for a comparison group are completed (i.e. assembled and passed QC) and are in the same directory before an adhoc cgMLST run is initiated [I noticed that the kraken2 rule ran after cgMLST rule...need to check sequentiality of the pipeline]

In [10]:
#output directory structure after incomplete snakemake run
#!pip install tree
!tree WGS-pipeline/output

WGS-pipeline/output
├── abricate
│   └── comparisonGroup1
│       ├── sample2
│       │   └── sample2.abricate.card.tsv
│       └── sample2.abricate.vfdb.tsv
├── AMR
│   └── comparisonGroup1
│       └── sample2.amrfinderplus.tsv
├── assembly
│   ├── comparisonGroup1
│   │   └── sample2_assembly
│   │       ├── contigs.fa
│   │       ├── shovill.corrections
│   │       ├── shovill.log
│   │       └── skesa.fasta
│   └── sample2
│       └── sample2
│           └── sample2_assembly
│               ├── contigs.fa
│               ├── shovill.corrections
│               ├── shovill.log
│               └── skesa.fasta
├── assemblyContigsOnly
│   └── comparisonGroup1
│       └── sample2_assembly_contigs.fa
├── fastq
│   └── sample2
│       ├── sample2.clean_fwd.fastq.gz
│       ├── sample2.clean_rev.fastq.gz
│       └── sample2.fastp.log.txt
├── mlst
│   └── comparisonGroup1
│       └── sample2
│           └── sample2.contigs.mlst.tsv
├── rmlst
│   └── comparisonGroup1
│       └── sample2.tsv


### Reran snakemake and completed it successfully
### A few points from completed snakemake run using only samples 1 and 2 (for simplicity)

1. `abricate` output: the folder structure should be consistent - for example: abricate/comparisonGroup1/card/sample1.abricate.card.tsv, abricate/comparisonGroup1/card/sample2.abricate.card.tsv, abricate/comparisonGroup1/vfdb/sample1.abricate.vfdb.tsv, abricate/comparisonGroup1/vfdb/sample2.abricate.vfdb.tsv

2. `AMR`finder output: is fine

3. `assembly` output: not sure why there are 2 directories for each sample? i.e. for sample1: assembly/comparisonGroup1/sample1 and assembly/sample1. There should only be one directory for each assembly, i.e. for sample1: assembly/comparisonGroup1/sample1

4. `assemblyContigsOnly` directory: this is fine, if this is an input for the adhoc cgMLST run, it should be in the appropriate directory for each comaprison group: e.g. cgMLST/comparisonGroup1/assemblyContigsOnly, cgMLST/comparisonGroup2/assemblyContigsOnly

5. `bracken` and `kraken` output: should these 2 directories be in a single directory, perhaps name it readsSpeciesID or something else

6. fastp output: the folder should be renamed to fastp or readsQC or something else

7. `mlst` and `rMLST` output: (A) is fine, though I'm not sure why you have a separate folder for each sample in the mlst directory, i.e. mlst/comparisonGroup1/sample1, mlst/comparisonGroup1/sample2; (B) Also, I think this should be in a folder called assemblySpeciesID

8. `sample_validation` output: I would place this output in a directory with the bracken and kraken output (as mentioned in point 5 above)

9. `trainingFiles` output: (A) the training file for chewBBACA should only be one per comparisonGroup (i.e. species) and is generated not from the samples, rather from a reference genome that is downloaded from NCBI (see the example that I provided here: https://github.com/itaysol/WGS-pipeline/issues/21#issuecomment-1794703642); (B) Also, the trainingFiles directory should be in the cgMLST directory, i.e. cgMLST/comparisonGroup1/trainingFile, cgMLST/comparisonGroup2/trainingFile

10. `wgkb` output: (A) not sure why there are 2 directories for each sample? i.e. for sample1: wgkb/comparisonGroup1/sample1 and wgkb/sample1. There should only be one directory for each assembly, i.e. for sample1: wgkb/comparisonGroup1/sample1; (B) Also, I think this should be in a folder called assemblySpeciesID, along with mlst and rMLST directories

11. `wgv` output: (A) not sure why each sample is in a folder, it only has a single file output correct? (B) Also, I think this should be in a folder called assemblySpeciesID, along with mlst, rMLST, wgkb directories

In [11]:
#output directory structure after snakemake run fully completed
!tree WGS-pipeline/output

WGS-pipeline/output
├── abricate
│   └── comparisonGroup1
│       ├── sample1
│       │   └── sample1.abricate.card.tsv
│       ├── sample1.abricate.vfdb.tsv
│       ├── sample2
│       │   └── sample2.abricate.card.tsv
│       └── sample2.abricate.vfdb.tsv
├── AMR
│   └── comparisonGroup1
│       ├── sample1.amrfinderplus.tsv
│       └── sample2.amrfinderplus.tsv
├── assembly
│   ├── comparisonGroup1
│   │   ├── sample1_assembly
│   │   │   ├── contigs.fa
│   │   │   ├── shovill.corrections
│   │   │   ├── shovill.log
│   │   │   └── skesa.fasta
│   │   └── sample2_assembly
│   │       ├── contigs.fa
│   │       ├── shovill.corrections
│   │       ├── shovill.log
│   │       └── skesa.fasta
│   ├── sample1
│   │   └── sample1
│   │       └── sample1_assembly
│   │           ├── contigs.fa
│   │           ├── shovill.corrections
│   │           ├── shovill.log
│   │           └── skesa.fasta
│   └── sample2
│       └── sample2
│           └── sample2_assembly
│               ├── contig